In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# Read in the atlantic data as lines and account for the different format of header lines
atlantic_lines = pd.read_csv('atlantic.txt', header=None, sep='\n')
atlantic_data = atlantic_lines[0].str.split(',', expand=True)

# Name the columns
cols1 = ['Date', 'Time', 'Record Identifier', 'Status', 'Latitude', 
         'Longitude', 'Max Wind', 'Min Pressure']
cols2 = ['Wind Radii Max Extent ' + str(x) for x in range(1, 13)]
cols = np.concatenate((cols1, cols2, ['is_header']))

# Clean up each dataframe
atlantic_data.columns = cols

# Fill the last column with boolean values that indicate if each line is a header line
atlantic_data.is_header = (atlantic_data.is_header != '')

# Filter out ancient storms that don't have real data
atlantic_data = atlantic_data.iloc[atlantic_data[atlantic_data.Date == 'AL012004'].index[0]:, :]

# Replace the missing values with actual NaN values, then impute the NaNs
atlantic_data.replace(' -999', np.nan, inplace=True)
atlantic_data.ffill(inplace=True)
atlantic_data.bfill(inplace=True)

# Get rid of N/W in Lat/Long
def fun(x):
        if x is not None:
            return x[:-1]
        else: 
            return x
atlantic_data['Latitude'] = pd.to_numeric(atlantic_data['Latitude'].apply(fun))
atlantic_data['Longitude'] = pd.to_numeric(atlantic_data['Longitude'].apply(fun))

# Convert these columns to numerical datatypes
num_cols = ['Max Wind', 'Min Pressure', 'Wind Radii Max Extent 1',
       'Wind Radii Max Extent 2', 'Wind Radii Max Extent 3',
       'Wind Radii Max Extent 4', 'Wind Radii Max Extent 5',
       'Wind Radii Max Extent 6', 'Wind Radii Max Extent 7',
       'Wind Radii Max Extent 8', 'Wind Radii Max Extent 9',
       'Wind Radii Max Extent 10', 'Wind Radii Max Extent 11',
       'Wind Radii Max Extent 12']
atlantic_data[num_cols] = atlantic_data[num_cols].apply(pd.to_numeric)

# One hot encode status
atlantic_data = pd.get_dummies(atlantic_data, prefix='Status', prefix_sep='', columns=['Status'], drop_first=True)

### FOR NOW ###
# Drop header record identifier
atlantic_data.drop(columns=['Record Identifier'], inplace=True)

# Separate and save a list of time series
atlantic_series_list = []
head_index = 0
for k in range(1, atlantic_data.shape[0]):
    
    if atlantic_data.iloc[k, :].is_header:
        tail_index = k - 1
        atlantic_series_list.append(atlantic_data.iloc[head_index:tail_index, :].drop(columns='is_header'))
        head_index = k
atlantic_series_list.append(atlantic_data.iloc[head_index:, :].drop(columns='is_header'))
pickle.dump(atlantic_series_list, open('atlantic_series.pickle', 'wb'))

In [3]:
# Read in the pacific data as lines and account for the different format of header lines
pacific_lines = pd.read_csv('pacific.txt', header=None, sep='\n')
pacific_data = pacific_lines[0].str.split(',', expand=True)

# Name the columns
cols1 = ['Date', 'Time', 'Record Identifier', 'Status', 'Latitude', 
         'Longitude', 'Max Wind', 'Min Pressure']
cols2 = ['Wind Radii Max Extent ' + str(x) for x in range(1, 13)]
cols = np.concatenate((cols1, cols2, ['is_header']))

# Clean up each dataframe
pacific_data.columns = cols

# Fill the last column with boolean values that indicate if each line is a header line
pacific_data.is_header = (pacific_data.is_header != '')

# Filter out ancient storms that don't have real data
pacific_data = pacific_data.iloc[pacific_data[pacific_data.Date == 'EP012004'].index[0]:, :]

# Replace the missing values with actual NaN values
pacific_data.replace(' -999', np.nan, inplace=True)

# Replace other typos found in the data and then fill nans
pacific_data.replace([' ST', ' TY', ' PT', ' WV'], np.nan, inplace=True)
pacific_data.ffill(inplace=True)
pacific_data.bfill(inplace=True)

# Get rid of N/W in Lat/Long
def fun(x):
        if x is not None:
            return x[:-1]
        else: 
            return x
pacific_data['Latitude'] = pd.to_numeric(pacific_data['Latitude'].apply(fun))
pacific_data['Longitude'] = pd.to_numeric(pacific_data['Longitude'].apply(fun))

# Convert these columns to numerical datatypes
num_cols = ['Max Wind', 'Min Pressure', 'Wind Radii Max Extent 1',
       'Wind Radii Max Extent 2', 'Wind Radii Max Extent 3',
       'Wind Radii Max Extent 4', 'Wind Radii Max Extent 5',
       'Wind Radii Max Extent 6', 'Wind Radii Max Extent 7',
       'Wind Radii Max Extent 8', 'Wind Radii Max Extent 9',
       'Wind Radii Max Extent 10', 'Wind Radii Max Extent 11',
       'Wind Radii Max Extent 12']
pacific_data[num_cols] = pacific_data[num_cols].apply(pd.to_numeric)

# One hot encode status
pacific_data = pd.get_dummies(pacific_data, prefix='Status', prefix_sep='', columns=['Status'], drop_first=True)

### FOR NOW ###
# Drop header record identifier
pacific_data.drop(columns=['Record Identifier'], inplace=True)

# Separate and save a list of time series
pacific_series_list = []
head_index = 0
for k in range(1, pacific_data.shape[0]):
    
    if pacific_data.iloc[k, :].is_header:
        tail_index = k - 1
        pacific_series_list.append(pacific_data.iloc[head_index:tail_index, :].drop(columns='is_header'))
        head_index = k
pacific_series_list.append(pacific_data.iloc[head_index:, :].drop(columns='is_header'))
pickle.dump(pacific_series_list, open('pacific_series.pickle', 'wb'))

In [4]:
atlantic = pickle.load(open('atlantic_series.pickle', 'rb'))
pacific = pickle.load(open('pacific_series.pickle', 'rb'))
print('Number of Atlantic Hurricanes/Time Series: ' + str(len(atlantic)))
print('Number of Pacific Hurricanes/Time Series: ' + str(len(pacific)))

Number of Atlantic Hurricanes/Time Series: 271
Number of Pacific Hurricanes/Time Series: 323


In [5]:
atlantic[40]

Date                 Time  Latitude  Longitude  Max Wind  \
46523  AL252005                WILMA      37.7        6.0        30   
46524  20051015                 1800      17.6       78.5        25   
46525  20051016                 0000      17.6       78.8        25   
46526  20051016                 0600      17.5       79.0        30   
46527  20051016                 1200      17.5       79.2        30   
46528  20051016                 1800      17.5       79.4        30   
46529  20051017                 0000      17.4       79.6        30   
46530  20051017                 0600      16.9       79.6        35   
46531  20051017                 1200      16.3       79.7        40   
46532  20051017                 1800      16.0       79.8        45   
46533  20051018                 0000      15.8       79.9        55   
46534  20051018                 0600      15.7       79.9        60   
46535  20051018                 1200      16.2       80.3        65   
46536  20051018                 1800      16.6       81.1        75   
46537  20051019                 0000      16.6       81.8       130   
46538  20051019                 0600      17.0       82.2       150   
46539  20051019                 1200      17.3       82.8       160   
46540  20051019                 1800      17.4       83.4       140   
46541  20051020                 0000      17.9       84.0       135   
46542  20051020                 0600      18.1       84.7       130   
46543  20051020                 1200      18.3       85.2       130   
46544  20051020                 1800      18.6       85.5       130   
46545  20051021                 0000      19.1       85.8       130   
46546  20051021                 0600      19.5       86.1       130   
46547  20051021                 1200      20.1       86.4       125   
46548  20051021                 1800      20.3       86.7       120   
46549  20051021                 2145      20.6       86.8       130   
46550  20051022                 0000      20.6       86.8       120   
46551  20051022                 0330      20.8       86.9       115   
46552  20051022                 0600      20.8       87.0       110   
46553  20051022                 1200      21.0       87.1       100   
46554  20051022                 1800      21.3       87.1        85   
46555  20051023                 0000      21.6       87.0        85   
46556  20051023                 0600      21.8       86.8        85   
46557  20051023                 1200      22.4       86.1        85   
46558  20051023                 1800      23.1       85.4        90   
46559  20051024                 0000      24.0       84.3        95   
46560  20051024                 0600      25.0       83.1       110   
46561  20051024                 1030      25.9       81.7       105   
46562  20051024                 1200      26.2       81.0        95   
46563  20051024                 1800      28.0       78.8       105   
46564  20051025                 0000      30.1       76.0       110   
46565  20051025                 0600      33.3       72.0       100   
46566  20051025                 1200      36.8       67.9        90   
46567  20051025                 1800      40.5       63.5        75   
46568  20051026                 0000      42.5       60.0        60   
46569  20051026                 0600      44.0       57.5        55   
46570  20051026                 1200      45.0       55.0        50   

       Min Pressure  Wind Radii Max Extent 1  Wind Radii Max Extent 2  \
46523          1003                        0                        0   
46524          1004                        0                        0   
46525          1004                        0                        0   
46526          1003                        0                        0   
46527          1003                        0                        0   
46528          1002                        0                        0   
46529          1001 

In [6]:
pacific[110]

Date                 Time  Latitude  Longitude  Max Wind  \
23600  EP162009                MARTY      19.9      136.4        20   
23601  20090915                 1200      17.3      111.3        25   
23602  20090915                 1800      17.7      111.6        25   
23603  20090916                 0000      18.0      111.9        25   
23604  20090916                 0600      18.3      112.2        30   
23605  20090916                 1200      18.6      112.4        35   
23606  20090916                 1800      18.8      112.5        40   
23607  20090917                 0000      19.0      112.6        40   
23608  20090917                 0600      19.2      112.7        40   
23609  20090917                 1200      19.4      112.8        40   
23610  20090917                 1800      19.6      113.0        40   
23611  20090918                 0000      19.9      113.4        35   
23612  20090918                 0600      20.1      113.9        35   
23613  20090918                 1200      20.5      114.4        35   
23614  20090918                 1800      21.0      114.9        30   
23615  20090919                 0000      21.5      115.6        30   
23616  20090919                 0600      22.1      116.5        30   
23617  20090919                 1200      22.4      117.5        25   
23618  20090919                 1800      22.7      118.5        25   
23619  20090920                 0000      22.8      119.6        25   
23620  20090920                 0600      22.9      120.6        25   
23621  20090920                 1200      23.0      121.6        25   
23622  20090920                 1800      23.1      122.6        25   
23623  20090921                 0000      23.1      123.7        25   
23624  20090921                 0600      23.1      124.8        20   
23625  20090921                 1200      23.1      125.9        20   
23626  20090921                 1800      23.1      126.9        20   
23627  20090922                 0000      23.1      127.9        20   
23628  20090922                 0600      23.2      129.1        20   
23629  20090922                 1200      23.3      130.4        20   

       Min Pressure  Wind Radii Max Extent 1  Wind Radii Max Extent 2  \
23600          1009                        0                        0   
23601          1007                        0                        0   
23602          1007                        0                        0   
23603          1007                        0                        0   
23604          1005                        0                        0   
23605          1003                        0                       50   
23606          1002                        0                       60   
23607          1002                        0                       45   
23608          1003                        0                       45   
23609          1003                       15                       45   
23610          1003                       15                       40   
23611          1004                       30                       40   
23612          1004                       30                       25   
23613          1004                       30                        0   
23614          1004                        0                        0   
23615          1004                        0                        0   
23616          1004                        0                        0   
23617          1005                        0                        0   
23618          1005                        0                        0   
23619          1005                        0                        0   
23620          1005                        0                        0   
23621          1006                        0                        0   
23622          1006                        0                        0   
23623          1007                        0             